# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import json

# Import API key
from apis_key import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tabou,4.4230,-7.3528,24.33,82,83,2.53,CI,1719641037
1,1,punta arenas,-53.1500,-70.9167,-0.94,86,75,3.09,CL,1719641039
2,2,utrik,11.2278,169.8474,27.94,83,97,7.02,MH,1719641040
3,3,chernyshevskiy,63.0128,112.4714,28.79,40,54,2.02,RU,1719641042
4,4,port saint john's,-31.6229,29.5448,17.79,49,7,3.16,ZA,1719641044


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# %%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 400,
    size = "Humidity",
    scale = 0.6,
    alpha = 0.5,
    color = "City")

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_ideal_df = city_data_df.loc[(city_data_df["Max Temp"] >=21) & (city_data_df["Max Temp"] <=26) & \
                                 (city_data_df["Wind Speed"] <10) & (city_data_df["Cloudiness"] <=50) & \
                                 (city_data_df["Humidity"] <=60)]

# Drop any rows with null values
city_ideal = city_ideal_df.dropna()

# Display sample data
city_ideal

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
70,70,kasumkent,41.6773,48.1460,22.86,45,27,3.13,RU,1719641162
73,73,zeya,53.7500,127.2667,24.72,21,7,3.77,RU,1719641167
81,81,shakhtersk,48.0625,38.5167,22.52,48,7,5.20,UA,1719641183
100,100,olyokminsk,60.3736,120.4264,25.34,22,0,0.65,RU,1719641215
106,106,pyhajarvi,67.0701,27.2176,25.01,55,43,3.98,FI,1719641226
152,152,aktau,43.6500,51.2000,25.32,59,3,5.13,KZ,1719641310
203,203,archangelos,36.2149,28.1149,25.99,46,0,3.44,GR,1719641402
207,207,vytegra,61.0064,36.4481,23.54,47,1,3.41,RU,1719641409
209,209,tahe,52.3333,124.7333,25.48,26,13,3.52,CN,1719641412
226,226,east london,-33.0153,27.9116,21.52,43,0,5.14,ZA,1719641444


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
70,kasumkent,RU,41.6773,48.1460,45,
73,zeya,RU,53.7500,127.2667,21,
81,shakhtersk,UA,48.0625,38.5167,48,
100,olyokminsk,RU,60.3736,120.4264,22,
106,pyhajarvi,FI,67.0701,27.2176,55,
152,aktau,KZ,43.6500,51.2000,59,
203,archangelos,GR,36.2149,28.1149,46,
207,vytegra,RU,61.0064,36.4481,47,
209,tahe,CN,52.3333,124.7333,26,
226,east london,ZA,-33.0153,27.9116,43,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kasumkent - nearest hotel: No hotel found
zeya - nearest hotel: Серебряный створ
shakhtersk - nearest hotel: No hotel found
olyokminsk - nearest hotel: Колос
pyhajarvi - nearest hotel: Hotelli Pyhätunturi
aktau - nearest hotel: Гульдана
archangelos - nearest hotel: Semina
vytegra - nearest hotel: Старый городъ
tahe - nearest hotel: No hotel found
east london - nearest hotel: No hotel found
arkhangel'sk - nearest hotel: Серафима
lazarevskoye - nearest hotel: Атлантис
antsohihy - nearest hotel: Escale
yreka - nearest hotel: Best Western Minner's Inn
novyy buh - nearest hotel: No hotel found
saint-leu - nearest hotel: Iloha Seaview Hôtel
ruidoso - nearest hotel: Hotel Ruidoso
cherepovets - nearest hotel: Rimini
nyurba - nearest hotel: Марина


,City,Country,Lat,Lng,Humidity,Hotel Name
70,kasumkent,RU,41.6773,48.1460,45,No hotel found
73,zeya,RU,53.7500,127.2667,21,Серебряный створ
81,shakhtersk,UA,48.0625,38.5167,48,No hotel found
100,olyokminsk,RU,60.3736,120.4264,22,Колос
106,pyhajarvi,FI,67.0701,27.2176,55,Hotelli Pyhätunturi
152,aktau,KZ,43.6500,51.2000,59,Гульдана
203,archangelos,GR,36.2149,28.1149,46,Semina
207,vytegra,RU,61.0064,36.4481,47,Старый городъ
209,tahe,CN,52.3333,124.7333,26,No hotel found
226,east london,ZA,-33.0153,27.9116,43,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
# %%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 400,
    size = "Humidity",
    scale = 1,
    alpha = 0.5,
    color = "City",
    hover_cols = ["Hotel Name", "Country"])

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)